In [13]:
# Import Library
import pandas as pd
import os
import re
import pymongo
from pymongo import MongoClient
from dotenv import load_dotenv

from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA

In [14]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGODB_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [15]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
    dimensions=1536
)

In [16]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq']

In [17]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
    text_key="text"
)

In [18]:
# Optimized Professional Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Resmi Allo Bank Indonesia.
    Tugas Anda adalah menjawab pertanyaan berdasarkan data FAQ resmi dari website Allo Bank.

    Tanggung jawab utama:
    Memberikan jawaban akurat dan informatif sesuai dengan kategori FAQ: umum, akun, produk, layanan, proses pengaduan nasabah, dan registrasi.
    Menyampaikan informasi dengan bahasa profesional, jelas, dan mudah dipahami.
    Memberikan panduan teknis (jika diperlukan) dalam format numerik atau langkah-langkah berurutan.
    Menanggapi keluhan atau kendala nasabah dengan solusi yang relevan dari FAQ.

    Aturan jawaban:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam konteks FAQ resmi.
    2. Jika pertanyaan tidak relevan atau tidak tercakup dalam FAQ, jawab dengan sopan bahwa Anda tidak memiliki informasi tersebut.
    3. Jangan mengarang jawaban atau memberikan informasi di luar dokumen sumber.
    4. Jangan mencantumkan link eksternal, tautan promosi, atau informasi tambahan di luar konteks.
    5. Gunakan format poin atau numerik bila menjelaskan prosedur atau langkah-langkah.

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [ ]:
# Model Configuration dengan GPT-4
llm_gpt4 = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=OPENAI_KEY,
    temperature=0,
    max_tokens=500
)

In [ ]:
# Optimized Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm_gpt4,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.78
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
    return_source_documents=True
)

def clean_answer(raw_answer):
    """Membersihkan jawaban dari referensi dan format khusus"""
    # Format daftar bernomor
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    # Hapus karakter khusus
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

def ask(query):
    try:
        # Proses query langsung tanpa validasi awal
        result = qa.invoke({"query": query})

        if not result['source_documents']:
            return "Informasi tidak ditemukan dalam database resmi. Silakan hubungi kami 080 4110 4110"

        # Pembersihan jawaban akhir
        return clean_answer(result['result'])

    except Exception as e:
        return f"Terjadi kesalahan sistem: {str(e)}\nSilakan coba lagi atau hubungi 080 4110 4110"

faq_df = pd.read_csv('/Users/ilham/OneDrive/Desktop/finalprojectrmt41/data/faq_allobank_clean.csv')

def answer_from_doc(question):
    row = faq_df[faq_df['question'] == question]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

# <b>Pertanyaan di dalam konteks FAQ Allo Bank secara Formal dan sesuai Database</b>

In [ ]:
# Test Cases 1
questions = [
    "Apa itu Allo Bank?",
    "Mengapa saya tidak bisa menggunakan Allo Pay saya?",
    "Mengapa saya tidak bisa log in di aplikasi Allo Bank?",
    "Apakah menyimpan dana di Allo Bank dipastikan aman?",
    "Mengapa aplikasi Allo Bank saya berjalan sangat lambat?",
    "Bagaimana jika saldo atau limit saya tidak cukup?",
    "Keuntungan apa yang saya dapatkan sebagai member Allo Bank?",
    "Mengapa saya tidak bisa mendaftarkan nomor ponsel/email di aplikasi Allo Bank?",
    "Biaya apa saja yang dikenakan ketika melakukan registrasi?",
    "Mengapa saya harus mengisi informasi pribadi saat proses registrasi?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Apa itu Allo Bank?
Jawaban:
Allo Bank adalah sebuah institusi perbankan yang terbentuk ketika Bank Harda Internasional (BHI) diakuisisi oleh Mega Corpora. Setelah diakuisisi, Bank Harda Internasional (BHI) secara resmi berganti nama menjadi PT. Allo Bank. Allo Bank didirikan di Indonesia pada tahun 2021.

ACTUAL ANSWER: Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan "All in One" dan mempunyai arti sebagai "one for all, and all for one" alias semua untuk satu dan satu untuk semua.

Pertanyaan: Mengapa saya tidak bisa menggunakan Allo Pay saya?
Jawaban:
Untuk dapat menggunakan Allo Pay, Anda perlu memastikan bahwa Anda telah melakukan proses registrasi Allo Pay. Jika Anda sudah melakukan registrasi tetapi masih mengalami kendala, mungkin ada masalah teknis yang perlu ditangani. Silakan hubungi 

Berdasarkan uji coba menggunakan model GPT-4 dengan pertanyaan FAQ resmi yang disampaikan secara formal dalam konteks Allo Bank, dan di mana pertanyaan tersebut memang tersedia dalam database, diketahui bahwa model mampu memberikan jawaban yang relevan, ringkas namun lebih to the point, dan tepat sesuai dengan informasi yang tersimpan di database.

# <b>Pertanyaan di dalam konteks FAQ Allo Bank secara Informal dan tidak sesuai Database</b>

In [22]:
# Test Case 2
questions = [
    "Allo Bank itu apa sih?",
    "Kenapa ya aku nggak bisa pakai Allo Pay?",
    "Aku nggak bisa login ke aplikasi Allo Bank, kenapa ya?",
    "Nyimpen uang di Allo Bank aman nggak sih?",
    "Kenapa aplikasi Allo Bank aku lemot banget?",
    "Kalau saldo atau limit aku nggak cukup, gimana ya?",
    "Kalau jadi member Allo Bank, aku dapet keuntungan apa aja sih?",
    "Kenapa aku nggak bisa daftar pakai nomor HP atau email di aplikasi Allo Bank?",
    "Pas daftar, ada biaya yang harus dibayar nggak sih?",
    "Kenapa sih aku harus isi data pribadi waktu daftar?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Allo Bank itu apa sih?
Jawaban:
Allo Bank adalah sebuah institusi perbankan yang terbentuk ketika Bank Harda Internasional (BHI) diakuisisi oleh Mega Corpora. Setelah diakuisisi, Bank Harda Internasional (BHI) secara resmi berganti nama menjadi PT. Allo Bank. Allo Bank didirikan di Indonesia pada tahun 
2021. Salah satu produk utamanya adalah Allo Prime, sebuah produk tabungan digital yang dapat diakses melalui aplikasi Allo Bank atau aplikasi Business Unit yang berada dibawah di naungan CT Corp.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Kenapa ya aku nggak bisa pakai Allo Pay?
Jawaban:
Untuk dapat menggunakan fitur Allo PayLater, Anda perlu memastikan bahwa Anda sudah login pada aplikasi Allo Bank dan menekan tombol "Allo PayLater" pada halaman beranda aplikasi. Jika Anda masih mengalami kendala, mungkin ada masalah teknis atau masalah lain yang perlu ditangani. Untuk mendapatkan bantuan lebih lanjut, silakan hubungi layanan pelanggan Allo Bank.

ACTUAL ANSWER: Tida

Dalam pengujian chatbot menggunakan model GPT-4, ditemukan bahwa meskipun pertanyaan yang diajukan bersifat informal dan tidak persis sama dengan pertanyaan yang ada di database FAQ, model tetap mampu memberikan jawaban yang relevan, to the point dan lebih akurat. Jawaban yang dihasilkan oleh model konsisten dengan informasi yang terdapat di database FAQ resmi.

Hal ini menunjukkan bahwa chatbot tidak hanya melakukan pencocokan persis teks pertanyaan, melainkan mampu memahami konteks dan makna dari pertanyaan pengguna. Dengan kata lain, model dapat menggeneralisasi dan menafsirkan pertanyaan dalam bahasa sehari-hari atau informal sehingga tetap mengacu pada informasi yang benar sesuai data FAQ.

Oleh karena itu, chatbot dapat memberikan jawaban yang sesuai dan membantu meskipun pertanyaan pengguna tidak ada secara eksplisit dalam database. Ini sesuai dengan tujuan utama dari proyek, yaitu membuat chatbot yang fleksibel dan ramah pengguna dengan kemampuan memahami pertanyaan dalam berbagai bentuk bahasa, tetapi tetap akurat berdasarkan sumber data resmi.

# <b>Pertanyaan di luar Konteks FAQ Allo Bank namun masih berkaitan dengan banking</b>

In [23]:
# Test Cases 3
questions = [
    "Bagaimana cara kerja sistem kredit di bank secara umum?",
    "Apa perbedaan antara rekening tabungan dan rekening giro?",
    "Bagaimana prosedur pengajuan KPR di bank lain selain Allo Bank?",
    "Apa itu suku bunga floating dan fixed dalam pinjaman?",
    "Bagaimana keamanan transaksi perbankan melalui aplikasi mobile banking?",
    "Apa yang dimaksud dengan layanan internet banking?",
    "Bagaimana cara mengajukan kartu kredit baru di bank konvensional?",
    "Apa saja biaya yang biasanya dikenakan pada rekening koran?",
    "Bagaimana cara cek mutasi rekening melalui ATM?",
    "Apa itu BI Checking dan bagaimana pengaruhnya terhadap pengajuan kredit?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Bagaimana cara kerja sistem kredit di bank secara umum?
Jawaban:
Maaf, saya tidak dapat memberikan informasi tentang cara kerja sistem kredit di bank secara umum karena informasi tersebut tidak termasuk dalam FAQ resmi Allo Bank. Saya hanya dapat memberikan informasi berdasarkan FAQ resmi Allo Bank. Untuk pertanyaan tersebut, saya sarankan Anda untuk menghubungi bank yang bersangkutan atau mencari informasi dari sumber yang terpercaya.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Apa perbedaan antara rekening tabungan dan rekening giro?
Jawaban:
Maaf, saya tidak dapat memberikan informasi tentang perbedaan antara rekening tabungan dan rekening giro karena informasi tersebut tidak tersedia dalam FAQ resmi Allo Bank. Saya menyarankan Anda untuk menghubungi layanan pelanggan Allo Bank untuk mendapatkan informasi yang lebih akurat dan lengkap. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Bagaimana prosedur pengajuan KPR di bank lain selain Allo Bank?
Jawaba

Berdasarkan pengujian model GPT-4 terhadap pertanyaan di luar konteks FAQ Allo Bank namun masih berkaitan dengan banking, diketahui bahwa model GPT-4 menunjukkan kemampuan yang lebih baik dibandingkan GPT-3.5-turbo dalam menolak menjawab pertanyaan umum perbankan yang berada di luar konteks FAQ Allo Bank dan database informasi resmi.

# <b>Pertanyaan di luar Konteks Allo Bank</b>

In [24]:
# Test Cases 4
questions = [
    "Siapa presiden dan wakil presiden pertama Indonesia?",
    "Kapan Indonesia merdeka?",
    "Apakah alien itu benar-benar ada?",
    "Bagaimana cara belajar coding untuk pemula?",
    "Siapa penemu bom atom?",
    "Siapa itu Oppenheimer?",
    "Siapa itu Christoper Nolan?",
    "Apa itu teori relativitas?",
    "Apakah black hole itu nyata?",
    "Mengapa bumi itu bulat?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Siapa presiden dan wakil presiden pertama Indonesia?
Jawaban:
Mohon maaf, sebagai asisten Allo Bank, saya hanya dapat memberikan informasi yang berkaitan dengan layanan dan produk Allo Bank. Untuk pertanyaan sejarah atau politik seperti ini, saya sarankan untuk mencarinya di sumber informasi yang terpercaya. Terima kasih atas pengertian Anda.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Kapan Indonesia merdeka?
Jawaban:
Mohon maaf, pertanyaan Anda tidak relevan dengan layanan dan informasi yang disediakan oleh Allo Bank. Kami hanya menyediakan informasi terkait layanan dan produk perbankan kami.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Apakah alien itu benar-benar ada?
Jawaban:
Maaf, pertanyaan Anda tidak relevan dengan layanan dan produk Allo Bank. Kami hanya dapat memberikan informasi berdasarkan FAQ resmi Allo Bank.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Bagaimana cara belajar coding untuk pemula?
Jawaban:
Maaf, pertanyaan Anda tidak relevan dengan FA

Berdasarkan pengujian model GPT-4 terhadap pertanyaan di luar konteks Allo Bank, diketahui bahwa model mampu menolak dengan cara yang sopan, ramah, dan profesional terhadap pertanyaan yang tidak relevan dan tidak terdapat dalam database FAQ. Selain itu, model juga memberikan informasi mengenai batasan konteks prompting yang digunakan.

# <b>Kesimpulan</b>

Berdasarkan serangkaian pengujian terhadap chatbot yang dibangun menggunakan model <b>GPT-4</b> dan berbasis pada database FAQ resmi Allo Bank, dapat disimpulkan bahwa:

1. <b>Kemampuan Memahami Variasi Pertanyaan</b>,
Chatbot mampu memberikan jawaban yang relevan, ringkas, dan lebih to the point terhadap pertanyaan resmi maupun informal yang masih sesuai dengan konteks FAQ Allo Bank. Model menunjukkan pemahaman konteks dan makna yang lebih baik dibandingkan model sebelumnya, sehingga jawaban yang diberikan lebih akurat dan jelas.

2. <b>Kemampuan Menjaga Batasan Konteks</b>,
Model GPT-4 lebih konsisten dalam menjaga batasan konteks proyek dengan menolak untuk menjawab pertanyaan umum perbankan yang berada di luar cakupan FAQ Allo Bank dan database resmi. Hal ini menunjukkan peningkatan kemampuan dalam mematuhi aturan konteks yang telah ditetapkan.

3. <b>Kemampuan Menolak dengan Tepat dan Profesional</b>,
Model mampu menolak pertanyaan yang tidak relevan dengan cara yang sopan, ramah, dan profesional, serta memberikan penjelasan mengenai batasan konteks prompting yang digunakan. Ini membuat chatbot lebih sesuai dengan tujuan proyek untuk memberikan informasi yang valid dan resmi saja.

<b>Model GPT-4</b>  

Kelebihan :  
- Dapat memberikan jawaban yang relevan, lebih singkat namun to the point, dan tetap akurat.
- Dapat memahami pertanyaan baik secara formal maupun informal.
- Lebih baik dalam menolak pertanyaan diluar konteks FAQ Allo Bank secara sopan dan profesional.
- Lebih konsisten dalam menjaga batasan konteks.

Kelemahan : 
- Lebih mahal daripada GPT-3.5-turbo.
- Waktu pemrosesan lebih lama.
- Jawaban terasa lebih kaku karena to the point